In [1]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")
import glob
import ee
import pickle
ee.Initialize()


In [2]:

def maskS2clouds(image):

  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask);
  

def NDWI_function(image):
  NDWI = image.expression(
      '(B03-B08)/(B03+B08)', {
        'B03': image.select('B3'),
        'B08': image.select('B8')

  }).rename('NDWI');
  return image.addBands(NDWI);

    
def TBVI1_function(image):

  TBVI1 = image.expression(
    'B05/(B05+B02)', {
      'B02': image.select('B2'),
      'B05': image.select('B5')

  }).rename('TBVI1');
  return image.addBands(TBVI1);


    
def NDVI_G_function(image):

  NDVI_G = image.expression(
  '(B8A-B03)/(B8A+B03)', {
  'B03': image.select('B3'),
  'B8A': image.select('B8A')

  }).rename('NDVI_G');
  return image.addBands(NDVI_G)

    
def SR_N_function(image):
  SR_N = image.expression(
  'B8A/B02', {
  'B02': image.select('B2'),
  'B8A': image.select('B8A')

  }).rename('SR_N');
  return image.addBands(SR_N);

def SR_n2_function(image):
  SR_n2 = image.expression(
      'B08/B03', {
        'B03': image.select('B3'),
        'B08': image.select('B8')
  
  }).rename('SR_n2');
  return image.addBands(SR_n2)

def NDVI_function(image):

  NDVI = image.expression(
      '(B08-B04)/(B08+B04)', {
        'B04': image.select('B4'),
        'B08': image.select('B8')

  }).rename('NDVI');
  return image.addBands(NDVI);

def maskS2clouds_min_max(image):
  
  qa = image.select('QA60');

  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  img = image.updateMask(mask).divide(10000);
  img_time = img.set("system:time_start", image.get("system:time_start"))
  return img_time

def addNDVI(image):
  ndvi_s = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  ndvi = ndvi_s.set("system:time_start", image.get("system:time_start"));
  return image.addBands(ndvi)

# def clipToCol(image):
#   return image.clipToCollection(point)


# def masker(image):
#   mask1 = greenest.select('NDVI').gt(0.3);
#   mask2 = min.select('NDVI').lt(0.6);
#   mask3=image.select('NDVI').gt(0.05)
#   mask4=image.select('NDVI').lt(0.3)
#   return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)



,index,Date,Latitude,Longitude
0,0,2021-03-3T09:12:31.094Z,33.800612,74.934720
1,1,2021-03-3T10:52:55.698Z,33.800849,74.808513
2,2,2021-03-3T12:12:58.064Z,33.800826,74.935571
3,3,2021-03-3T13:08:16.906Z,33.800448,74.935790
4,4,2021-03-3T14:21:03.017Z,33.800858,74.935616
...,...,...,...,...
9794,9794,2021-03-17T18:42:57.667Z,17.866610,78.670140
9795,9795,2021-03-17T18:51:22.884Z,17.866590,78.673250
9796,9796,2021-03-17T02:26:01.523Z,17.813700,78.700730
9797,9797,2021-03-17T06:06:18.324Z,17.810820,78.685370


In [20]:
df = pd.read_csv("../lat_lons.csv").reset_index()

points_lmt = 2000

df.loc[:points_lmt, 'Date'] = pd.to_datetime(df.loc[:points_lmt,'Date'].apply(lambda x: x.split("T")[0]), format = '%Y-%m-%d')

df_tmp = df.loc[:points_lmt, :]

In [19]:

def masker(greenest, min):
    def wrap(image):                
        mask1 = greenest.select('NDVI').gt(0.3);
        mask2 = min.select('NDVI').lt(0.6);
        mask3=image.select('NDVI').gt(0.05)
        mask4=image.select('NDVI').lt(0.3)
        return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)
    return wrap

def clipToCol(point):
    def wrap(image):
            
        return image.clipToCollection(point)
    return wrap

def predictor_time(row):
    
    current_date = row['Date']

    start_date = ee.Date(current_date - datetime.timedelta(days = 16))

    end_date= ee.Date(current_date)
    point = ee.FeatureCollection(ee.Geometry.Point([row['Longitude'], row['Latitude']]));

    s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")

    s2_sr_filter = s2_sr.filterBounds(point).filterDate(start_date,end_date).map(maskS2clouds);
                    
    sentinel2 = ee.ImageCollection(s2_sr_filter.mosaic().set("system:time_start", ee.Image(s2_sr_filter.first()).get("system:time_start")))

    indices_collection = sentinel2.map(NDWI_function).map(TBVI1_function).map(NDVI_G_function).map(SR_N_function).map(SR_n2_function).map(NDVI_function);
    

    s2_NDVI = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(point).filterDate('2020-01-01', '2020-12-31')\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50)).map(maskS2clouds_min_max).map(addNDVI).select("NDVI").map(clipToCol(point));

    greenest = s2_NDVI.qualityMosaic('NDVI')
    min = s2_NDVI.min()

    predictor_bands = indices_collection.map(masker(greenest, min))

    df = pd.DataFrame(predictor_bands.select('B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G')\
        .getRegion(point, 10).getInfo())
    
    df, df.columns = df[1:] , df.iloc[0] 
    df = df.drop(["id"], axis =1)

    return df.values[0]


# asap = df_tmp.apply(predictor_time, axis = 1, result_type='expand', )

In [21]:
df_tmp = df_tmp.dropna()

In [23]:
asap = df_tmp.apply(predictor_time, axis = 1,  result_type='expand')


In [24]:
asap

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,75.819023,26.915547,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,77.541363,26.747921,1615354891931,2400.0,1834.0,1981.0,2839.0,2711.0,1078.0,1.570681,1.905956,0.608228,-0.221996,0.228283
1999,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
asap['sys_index'] = df_tmp['index']

In [25]:
asap.columns = ['longitude', 'latitude', 'time' ,'B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G']

In [26]:
asap

,longitude,latitude,time,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,75.819023,26.915547,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,77.541363,26.747921,1615354891931,2400.0,1834.0,1981.0,2839.0,2711.0,1078.0,1.570681,1.905956,0.608228,-0.221996,0.228283
1999,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
soil_nuts = [ 'P', 'K', 'OC', 'N', 'pH']

input = asap[['B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G']]

mylist = []
asd = input.dropna()
for i in soil_nuts:
    
    for nut, nut_slr in zip(glob.glob(f'../data/{i}.pkl'), glob.glob(f'../data/{i}*_slr.pkl')):
        with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

            mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
            X_test = model_scaler.transform(asd)
            
            predictions = mymodel.predict(X_test).reshape(-1,)
            if i == 'OC':
                predictions = predictions.clip(0, 1)
            mylist.append(predictions)
mydf = pd.DataFrame(mylist).T
mydf.columns = [ 'P', 'K', 'OC', 'N', 'pH']
mydf.index = asd.index

# pd.concat([asd, mydf], axis = 1, ignore_index=True,)
pd.concat([asap, mydf], axis = 1, ignore_index=False) 


,longitude,latitude,time,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G,P,K,OC,N,pH
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492,12.241877,716.042274,0.202540,265.840437,7.9550
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,75.819023,26.915547,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,77.541363,26.747921,1615354891931,2400.0,1834.0,1981.0,2839.0,2711.0,1078.0,1.570681,1.905956,0.608228,-0.221996,0.228283,8.993538,172.495412,0.309956,190.875999,8.0375
1999,75.778958,26.922105,1615354898700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
pd.concat([asap, mydf], axis = 1, ignore_index=False).to_csv("../first2000.csv")

In [ ]:
# soil_nuts = ['pH', 'P', 'K', 'OC', 'N']
# df_tmp = df[["longitude", "latitude"]]

# input_bands = ['B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G']
# df[input_bands] = df[input_bands].apply(lambda x: x.replace("None", np.nan))


# df_pred = df.loc[df['NDWI'].notna(), input_bands]

# input = df_pred.values

# for i in soil_nuts:
    
#   for nut, nut_slr in zip(glob.glob(f'../data/{i}.pkl'), glob.glob(f'../data/{i}*_slr.pkl')):
#     # print(i, nut, nut_slr)
#     with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

#       mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
#       X_test = model_scaler.transform(input)
#       predictions = mymodel.predict(X_test)
#       print(predictions, i)